# In-class tasks

Install RxPY and run all of the examples in this slide set (download the file lambtest.txt from the moodle)

In [1]:
import rx
from rx import operators as ops
from rx import create
from rx import from_

def push_five_strings(observer, scheduler):
    observer.on_next("Alpha")
    observer.on_next("Beta")
    observer.on_next("Gamma")
    observer.on_next("Delta")
    observer.on_next("Epsilon")
    observer.on_completed()

class PrintObserver:
    def on_next(self, value):
        print("Received {0}".format(value))
        
    def on_completed(self):
        print("Done!")
    
    def on_error(self, error):
        print("Error Occurred {0}".format(error))
        
source = create(push_five_strings)
source.subscribe(PrintObserver())

Received Alpha
Received Beta
Received Gamma
Received Delta
Received Epsilon
Done!


In [2]:
source = from_(["A", "B", "C", "D", "E"])
source.subscribe(on_next=lambda value: print("Received {0}".format(value)),
                on_completed=lambda: print("Done!"),
                on_error=lambda error: print("Error Occurred {0}".format(error))
                )

Received A
Received B
Received C
Received D
Received E
Done!


In [3]:
from_(["Alpha", "Beta", "Gamma", "Delta", "Ro"]).pipe(
    ops.map(lambda s: len(s)),
    ops.filter(lambda i: i >= 5)
).subscribe(lambda value: print("Received {0}".format(value)))

Received 5
Received 5
Received 5


In [4]:
from_(["one two", "three four", "five six"]).pipe(
    ops.take(2),
    ops.flat_map(lambda s: s.split())
).subscribe(lambda value: print("{0}".format(value)))

one
two
three
four


In [5]:
file = open('lambtest.txt')
o = from_(file).pipe(
    ops.filter(lambda i: i[0] == 'T')
)
o.subscribe(lambda s: print(s))

The Sages say truly

That two animals are in this forest:

The other an unicorn.

They are concealed in the forest,

The Masters shew you here clearly

That in all places

These two animals wander about in forests

That will be rightly and truly said.

The unicorn will be the Spirit at all times.

The deer desires no other name

To couple them together,

That he has attained the golden flesh,



In [6]:
o3 = rx.interval(1).pipe(
    ops.map(lambda i: i),
    ops.take_while(lambda i: i < 10)
)
o3.subscribe(lambda s: print(s))

0
1
2
3
4
5
6
7
8
9


In [7]:
from_([1,2,3,4,5,21,22,23,200,300]).pipe(
    ops.filter(lambda i: i < 20),
    ops.reduce(lambda t, i: t + i)
).subscribe(lambda s: print(s))

15


In [8]:
from_(["A", "B", "C", "D"]).pipe(
    ops.to_list()
).subscribe(lambda i: print(i))

['A', 'B', 'C', 'D']


In [9]:
L = from_(["a", "b", "c", "d", "e"])
N = from_([1,2,3,4,5])

L.pipe(ops.merge(N)).subscribe(lambda s: print(s))

a
b
c
d
e
1
2
3
4
5


In [10]:
o1 = rx.interval(0.3).pipe(ops.map(lambda i: "O1 {0}".format(i)))
o2 = rx.interval(0.5).pipe(ops.map(lambda i: "O2 {0}".format(i)))
o3 = rx.interval(1).pipe(ops.map(lambda i: "O3 {0}".format(i)))

o1.pipe(ops.merge(o2, o3)).subscribe(lambda s: print(s))

O1 0
O2 0
O1 1
O1 2
O3 0
O2 1
O1 3
O1 4
O2 2
O1 5
O3 1

In [3]:
o1 = rx.interval(3).pipe(ops.map(lambda i: "O1 {0}".format(i)))
o2 = rx.interval(5).pipe(ops.map(lambda i: "O2 {0}".format(i)))
o3 = rx.interval(1).pipe(ops.map(lambda i: "O3 {0}".format(i)))
olist = [o1, o2, o3]

from_(olist).pipe(ops.merge_all()).subscribe(lambda s: print(s))

O3 0
O3 1
O1 0
O3 2
O3 3
O2 0
O3 4
O1 1
O3 5


In [2]:
from_(["one two", "three four", "five six"]). pipe(
    ops.map(lambda s: from_(s.split())),
    ops.merge_all()
).subscribe(lambda s: print(s))

one
two
three
four
five
six


In [3]:
L = from_(["a", "b", "c", "d", "e"])
N = from_([1,2,3,4,5])
L.pipe(ops.concat(N)).subscribe(lambda s: print(s))

a
b
c
d
e
1
2
3
4
5


In [4]:
o3 = rx.interval(1).pipe(ops.map(lambda i: i),
                        ops.take_while(lambda i: i < 10))
o5 = from_(["A", "B", "C"])
o3.pipe(ops.concat(o5)).subscribe(lambda s: print(s))

0
1
2
3
4
5
6
7
8
9
A
B
C


In [5]:
L = from_(["a", "b", "c", "d", "e", "f", "g"])
N = from_([1,2,3,4,5])
L.pipe(ops.zip(N)).subscribe(lambda s: print(s))

('a', 1)
('b', 2)
('c', 3)
('d', 4)
('e', 5)


In [2]:
from_(["Alpha", "Beta", "Gamma", "Delta", "Ro"]).pipe(
    ops.group_by(lambda s: len(s)),
    ops.flat_map(lambda g: g.pipe(ops.to_list())),
).subscribe(lambda s: print(s))

['Alpha', 'Gamma', 'Delta']
['Beta']
['Ro']


# Simple word count

In [7]:
file = open('lambtest.txt')

source = from_(file).pipe(ops.flat_map(lambda s: s.split()), 
                          ops.count()
)

source.subscribe(lambda value: print("Number of words: {0}".format(value)))

Number of words: 174


In [3]:
import re
o = from_(open('lambtest.txt')).pipe(ops.map(lambda l: l.lower()),
                                     ops.map(lambda w: re.sub(r'[^\w\s]','', w)),
                                     ops.flat_map(lambda s: s.split()),
                                     ops.distinct(),
                                     ops.group_by(lambda i: i[0]),
                                     ops.flat_map(lambda g: g.pipe(ops.to_list())),
                                     ops.to_dict(lambda x: x[0][0], lambda x: len(x))
)
o.subscribe(lambda val: print(val))

{'t': 14, 's': 10, 'a': 13, 'i': 4, 'f': 5, 'o': 6, 'g': 3, 'b': 6, 'd': 2, 'u': 1, 'c': 6, 'h': 5, 'm': 4, 'w': 5, 'y': 1, 'p': 2, 'k': 2, 'r': 2, 'n': 3, 'l': 1, 'j': 2, 'e': 1}
